# Exercise 3

## Question 1
Load the prices data from the CSV file to the SQLite database.

In [ ]:
import sqlite3
import csv
from contextlib import closing

conn = sqlite3.connect("Exercise3.db")
cs = conn.cursor()
cs.execute("""
create table if not exists prices (
theday text primary key,
price real
);
""")
with closing(open('SP500.csv')) as datafile:
    reader = csv.DictReader(datafile, fieldnames=["date", "price"], delimiter='\t')
    for row in reader:
        cs.execute(F"insert into prices values (\"{row['date']}\", {float(row['price'])})")
conn.commit()


## Question 2
Calibrate a GBM model to the prices of the 120 days prior to *2021-05-31*.

**Hint**:
1. Read the data from the database
2. Calibrate the GBM model in exactly the same way as in Question 3 of Exercise 2.

In [ ]:
import numpy as np
import sqlite3
import csv
from contextlib import closing

class GBM:
    def __init__(self):
        self.mu = np.nan;
        self.sigma = np.nan;
        self.rng = np.random.default_rng()

    def calibrate(self, trajectory, Dt):
        increments = np.diff(np.log(trajectory));
        moments = [0, 0];
        n_iter = 10;
        for iter in range(n_iter):
            X = self.rng.choice(increments, size=len(increments)//2)
            moments[0] += np.mean(X)/n_iter;
            moments[1] += np.mean(X**2)/n_iter
        std = np.sqrt(moments[1] - moments[0]**2);
        self.sigma = std/np.sqrt(Dt);
        self.mu = moments[0] / Dt + self.sigma**2/2;

test_date = '2021-05-31'
N = 120
model = GBM();
P = None
with closing(sqlite3.connect("Exercise3.db")) as conn:
    cs = conn.cursor()
    cs.execute(F"""
    select price from prices
    where theday < '{test_date}'
    order by theday desc
    limit {N};
    """)
    P = np.flipud(np.asarray(cs.fetchall()))
    model.calibrate(P.flat, 1/250);

print(F"Model parameters: % .4f,  % .4f" % (model.mu, model.sigma))


Model parameters:  0.0511,   0.1440


## Question 3
Use the calibrated GBM model to forecast the price on *2021-05-31*. What is the 95% confidence interval of the forecast?

**Hint**

Recall the formula for price forecast:
$$S_{t + \Delta t} = S_t \exp \left[
\left(
\mu - \frac{\sigma^2}{2}
\right)\Delta t + \sigma \left(
W_{t + \Delta t} - W_t
\right)
\right]
$$

Our price forecast is
$$
\mathbb E S_{t+\Delta t} = S_t \exp \left(
\mu \Delta t
\right)
$$

$S_t$ is the latest price we have, i.e. the price on *2021-05-30*. $\Delta t = 1/250$ - make sure you use the same $\Delta t$ in forecasting as you have done in calibration. $(\mu - \frac{\sigma^2}{2})t + \sigma(W_{t+\Delta t} - W_{t})$ has normal distribution with mean $(\mu - \frac{\sigma^2}{2})\Delta t$ and standard deviation $\sigma \sqrt{\Delta t}$. Use these parameters with scipy.stats.norm.ppf to calculate the 2.5% quantile and the 97.5% quantile of the distribution. Call them $q_1$ and $q_2$.

Then the required confidence interval is $(S_t e^{q_1}, S_t e^{q_2})$.

In [ ]:
from scipy.stats import norm

Dt = 1/250
forecast = (P[-1] * np.exp(model.mu * Dt))[0]
print(F"Price forecast: {forecast}")

Q = norm.ppf([2.5e-2, 97.5e-2],
             loc=(model.mu - model.sigma**2/2)*Dt,
             scale=model.sigma*np.sqrt(Dt))
confidence_interval = P[-1] * np.exp(Q)
print(F"Confidence Intervals: % .4f,  % .4f" % (confidence_interval[0], confidence_interval[1]))


Price forecast: 4210.960080171914
Confidence Intervals:  4136.3056,   4286.6066
